## Названия переменных с итоговыми таблицами:
**result_tail** - таблица остатков для начальника

**df_tail** - таблица остатков для корректировки

**total_fin** - общая таблица финансирования по ТОВД

**fin_tovd** - таблица финансирования для конкретного ТОВД

**fin_oumts**- таблица финансирования по ОУМТСам

## Подготовка к написанию

**А. Вводится переменная-список *all_failes* для всех файлов-распределений;**

**В. Импортируем Библиотеку Панд, НаН из Нампая на будущее и ОС;**

**С. Вводится функция *cleaner*, обрабатывающая таблицу в нужный для кода формат:**

*С.1.* Автоматически проставляю соответствующие задания ГОЗ в каждой строке;

*С.2.* Автоматически проставляю соответствующие КБК в каждой строке;

*С.3.* Превращаю КБК в object.

*Добавить срез по КБК, если понадобится*

In [17]:
import pandas as pd
from numpy import nan
from os import listdir
from os.path import isfile, join
def cleaner(tabl):
    tabl['Задание ГОЗ'] = tabl['Задание ГОЗ'].fillna(method='ffill')
    tabl['КБК'] = tabl['КБК'].fillna(method='ffill')
    tabl['КБК'] = tabl['КБК'].astype('object')
    return tabl

## 1. Открываем все таблицы в переменные

**1.1** Просто непонятный цикл из чужой библиотеки по открытию;

**1.2** Подготавливаем все файлы к конкату и добавляем номера писем.

In [18]:
#Переменная для обновления списка (чтобы не задвоилось/троилось)
all_files = []
path = 'Письма'
all_name_files = [f for f in listdir(path) if isfile(join(path, f)) and f.endswith(".xlsx")]


for name in all_name_files:
    source = pd.read_excel(path+"/"+name, header=6)
    del source['Итого:']
    source = source.drop([len(source)-1])
    source = cleaner(source)
    source ['Номера писем'] = [name[:-5]]*len(source)
    all_files.append(source)

### 2. Объединение всех распределений

*Происходит по столбцам с заданиями ГОЗ, наименованиям закупки и КБК.* **(Нужно удалить в этом окне переменную frames. Она определена еще в первом действии)**

In [24]:
cat = pd.concat(all_files)

## 3. Группировка объединенных таблиц

*Практически готовая таблица по финансированию тер органов в переменной df_pivo.*

**3.1** Осущестляется группировка по заданиям ГОЗ, КБК и наименованиям закупки в переменно дф_пиво;

**3.2** Апгрейд дф_пива и добавление столбца "распределение/отзыв" с суммой распределений и отзывов по всем ТОВД.

In [25]:
df_pivo = cat.pivot_table(index=['Задание ГОЗ', 'Наименование закупки', 'КБК'], values=cat.columns[3:], aggfunc='sum', sort=False)
#Это pivo для Вани
df_pivo.loc[:, 'Распределение/отзыв'] = 0
tovd_all = df_pivo.columns[:-1]
df_pivo['Распределение/отзыв'] = df_pivo[tovd_all].agg('sum', axis="columns")



## 4. Первые итоговые таблицы *ОСТАТКОВ*.

**4.1** Создание промежуточной таблицы с движением ЛБО по наименованиям закупки и КБК в переменной *df_traffic* со столбцами ЛБО и Кол-во, группируем;

**4.2** Открываем предварительно созданную таблицу по запланированным/откорректированным ЛБО в переменной *lbo* и обрабатываем функцией клинера. Группируем по заданиям ГОЗ, наименованиям закупки и КБК;

**4.3** **Создание итоговой таблицы №1 для корректировок в переменной *df_tail*:**

    4.3.1 Объединяем таблицу трафика и ЛБО;
    4.3.2 Создаем функцию *tail_calc* для подсчета остатков (ЛБО-Распределения/отзывы);
    4.3.3 Создаем столбик остатков и применяем к нему функцию *tail_calc*;
    4.3.4 Создаем столбик с остатками исключительно по заданиям ГОЗ (без наименования закупки), заполняем его через цикл построчно.
**4.4** **Создаем вторую итоговую таблицу №2 для начальства в переменной *result_tail*:**

    4.4.1 Удаляем ненужные столбцы;
    4.4.2 Обнуляем группировку через reset_index;
    4.4.3 Группируем по-новому, как надо и переименовываем стобцы.

In [34]:
#1
df_traffic = df_pivo['Распределение/отзыв'].to_frame(name='Распределение/отзыв')
df_traffic.loc[:, "Кол-во"] = 0
df_traffic.loc[:, "ЛБО"] = 0
#2
lbo = pd.read_excel('Лимиты 2023 211.xlsx')
lbo = cleaner(lbo)
lbo = lbo.pivot_table(index=['Задание ГОЗ', 'Наименование закупки', 'КБК'], values=lbo.columns[3:], sort=False)
#3.1
df_tail = pd.concat([df_traffic, lbo])
df_tail = df_tail.pivot_table(index=['Задание ГОЗ', 'Наименование закупки', 'КБК'], values=['Распределение/отзыв', 'Кол-во', 'ЛБО'], aggfunc='sum', sort=False)

#3.2
# Функция для подсчета остатков:
def tail_calc (row):
    return row['ЛБО']-row['Распределение/отзыв'] 

#3.3
df_tail['Остатки'] = df_tail.apply(tail_calc, axis=1)
df_tail

#3.4
df_tail.loc[:, 'Сгруппированные остатки'] = nan
for group in df_tail.index.get_level_values(0).unique():
    namepurch = df_tail.loc[[group]].index[0][1]
    kbk = df_tail.loc[[group]].index[0][2]
    df_tail.loc[(group, namepurch, kbk), 'Сгруппированные остатки'] = df_tail.loc[[group], 'Остатки'].sum()

#4.1
result_tail = df_tail.droplevel(level=1)
del result_tail['Кол-во']
del result_tail['ЛБО']
del result_tail ['Распределение/отзыв']

#4.2
result_tail = result_tail.reset_index()

#4.3
result_tail = result_tail.groupby(['Задание ГОЗ', 'КБК'], sort=False) ['Сгруппированные остатки', 'Остатки'].sum()
result_tail = result_tail.rename (
columns={
    'Сгруппированные остатки':'Остатки по заданию ГОЗ',
    'Остатки':'Остатки по КБК',
}
)

C:\Users\79990\AppData\Local\Temp\ipykernel_6016\729567240.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_tail.loc[(group, namepurch, kbk), 'Сгруппированные остатки'] = df_tail.loc[[group], 'Остатки'].sum()
C:\Users\79990\AppData\Local\Temp\ipykernel_6016\729567240.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_tail.loc[(group, namepurch, kbk), 'Сгруппированные остатки'] = df_tail.loc[[group], 'Остатки'].sum()
C:\Users\79990\AppData\Local\Temp\ipykernel_6016\729567240.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_tail.loc[(group, namepurch, kbk), 'Сгруппированные остатки'] = df_tail.loc[[group], 'Остатки'].sum()
C:\Users\79990\AppData\Local\Temp\ipykernel_6016\729567240.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_tail.loc[(group, namepurch, kbk), 'Сгруппированные остатки'] = df_tail.loc[[group], 'Остатки'].sum()
C:\Users\79990\AppData\L

Кол-во   ЛБО  Распределение/отзыв  \
Задание ГОЗ Наименование закупки КБК                                        
Группа А    А (тип 1)            302.0     100  1300                 88.0   
            А (тип 2)            302.0     100  1300                191.0   
            А (тип 3)            302.0     100  1400                230.0   
            А (тип 4)            302.0       0     0               1258.0   
Группа В    В (тип 1)            302.0     100  1100                 42.0   
            В (тип 2)            302.0     100  1000                 40.0   
            В (тип 3)            302.0     100   500                 60.0   
            В (тип 4)            302.0     100  1000                 60.0   
Группа С    С (тип 1)            302.0     100  2000                 31.0   
            С (тип 2)            706.0     100   100                 31.0   
            С (тип 3)            302.0     100   200                 88.0   
            С (тип 4)            302.0     100   200                 32.0   
            С (тип 5)            302.0     100   200                 17.0   
Группа D    D (тип 1)            302.0     100   200                 48.0   
            D (тип 2)            302.0     100   200                 40.0   
            D (тип 3)            302.0     100   200                 60.0   
            D (тип 4)            302.0     100   200                 58.0   
Группа E    С (тип 1)            302.0     100   200                 35.0   
            С (тип 2)            706.0     100   200                 46.0   
            С (тип 3)            302.0     100   200                 91.0   
            С (тип 4)            302.0     100   200                 40.0   
            С (тип 5)            302.0     100   200                 17.0   
            С (тип 4)            706.0     100   200                  0.0   

                                        Остатки  Сгруппированные остатки  
Задание ГОЗ Наименование закупки КБК                                      
Группа А    А (тип 1)            302.0   1212.0                   2233.0  
            А (тип 2)            302.0   1109.0                      NaN  
            А (тип 3)            302.0   1170.0                      NaN  
            А (тип 4)            302.0  -1258.0                      NaN  
Группа В    В (тип 1)            302.0   1058.0                   3398.0  
            В (тип 2)            302.0    960.0                      NaN  
            В (тип 3)            302.0    440.0                      NaN  
            В (тип 4)            302.0    940.0                      NaN  
Группа С    С (тип 1)            302.0   1969.0                   2501.0  
            С (тип 2)            706.0     69.0                      NaN  
            С (тип 3)            302.0    112.0                      NaN  
            С (тип 4)            302.0    168.0                      NaN  
            С (тип 5)            302.0    183.0                      NaN  
Группа D    D (тип 1)            302.0    152.0                    594.0  
            D (тип 2)            302.0    160.0                      NaN  
            D (тип 3)            302.0    140.0                      NaN  
            D (тип 4)            302.0    142.0                      NaN  
Группа E    С (тип 1)            302.0    165.0                    971.0  
            С (тип 2)            706.0    154.0                      NaN  
            С (тип 3)            302.0    109.0                      NaN  
            С (тип 4)            302.0    160.0                      NaN  
            С (тип 5)            302.0    183.0                      NaN  
            С (тип 4)            706.0    200.0                      NaN

## 6. Вторые итоговые таблицы *ФИНАНСИРОВАНИЯ*.

**6.1** Вводится переменная *total_fin*, в которую добавляется строчка "Итог:" с подсчетами;

**6.2** Полученная таблица группируется по Заданиям ГОЗ, Наименованиям закупки и КБК.


In [35]:
#1
total_fin = cat.append({'Задание ГОЗ':'Итог:', 'Наименование закупки':' ', 'КБК':' '}, ignore_index=True)
total = list(total_fin[total_fin.columns[3:-1]].sum())
total_fin.loc[len(total_fin)-1, total_fin.columns[3:-1]] = total

#2
total_fin = total_fin.pivot_table(index=['Задание ГОЗ', 'Наименование закупки', 'КБК'], values=total_fin.columns[3:], aggfunc='sum', sort=False)

C:\Users\79990\AppData\Local\Temp\ipykernel_6016\1035356381.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_fin = cat.append({'Задание ГОЗ':'Итог:', 'Наименование закупки':' ', 'КБК':' '}, ignore_index=True)


In [36]:
tovd = ''

fin_tovd = pd.concat(all_files, axis=0,join='outer')

fin_tovd

,Задание ГОЗ,Наименование закупки,КБК,ВОГОиП МВД России,МВД по Республике Адыгея,МВД по Республике Алтай,МВД по Республике Башкортостан,МВД по Республике Бурятия,МВД по Республике Дагестан,МВД по Республике Ингушетия,...,ФКУ ГЦАХиТО МВД России,ОПБ МВД России,БСТМ МВД России,ЦСН БДД МВД России,ФКУ ЦКО МВД России,ФКУ ЦСР МВД России,"ФКУ ""ЦВР МВД России, с. Пересыпкино-2""",Распорядительный счет МВД России,ДТ МВД России,Номера писем
0,Группа А,А (тип 1),302.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test1
1,Группа А,А (тип 2),302.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,Test1
2,Группа А,А (тип 3),302.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test1
3,Группа А,А (тип 4),302.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Test1
4,Группа В,В (тип 1),302.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,Test1
5,Группа В,В (тип 2),302.0,NaN,NaN,NaN,NaN,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test1
6,Группа В,В (тип 3),302.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,Test1
7,Группа В,В (тип 4),302.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test1
8,Группа С,С (тип 1),302.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,Test1
9,Группа С,С (тип 2),706.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Test1
